# Economic way to graph a simple moving average

In [ ]:
from collections import deque
import pandas as pd
from datetime import datetime

class StreamingMovingAverage:
    """
    Efficient streaming moving average calculator that maintains state
    and calculates MA incrementally as new data points arrive.
    
    Uses a circular buffer (deque) for O(1) operations and running sum
    to avoid recalculating the entire average each time.
    """
    
    def __init__(self, window_size):
        """
        Initialize the streaming moving average calculator.
        
        Parameters:
        -----------
        window_size : int
            Number of data points to include in the moving average
        """
        self.window_size = window_size
        self.buffer = deque(maxlen=window_size)  # Circular buffer with fixed size
        self.sum = 0.0  # Running sum for O(1) average calculation
        self.timestamps = deque(maxlen=window_size)  # Store corresponding timestamps
        
    def add_data_point(self, timestamp, price):
        """
        Add a new data point and calculate the updated moving average.
        
        Parameters:
        -----------
        timestamp : datetime or str
            Timestamp of the data point
        price : float
            Price value
            
        Returns:
        --------
        dict
            Dictionary containing current MA, count of data points, and metadata
        """
        # Convert timestamp to datetime if it's a string
        if isinstance(timestamp, str):
            timestamp = pd.to_datetime(timestamp)
        
        # Check if buffer is full and we need to remove the oldest value
        if len(self.buffer) == self.window_size:
            # Remove the oldest value from the sum
            oldest_price = self.buffer[0]  # This will be automatically removed by deque
            self.sum -= oldest_price
        
        # Add the new price to buffer and sum
        self.buffer.append(price)
        self.timestamps.append(timestamp)
        self.sum += price
        
        # Calculate current moving average
        current_ma = self.sum / len(self.buffer)
        
        return {
            'timestamp': timestamp,
            'price': price,
            'moving_average': current_ma,
            'data_points': len(self.buffer),
            'is_full_window': len(self.buffer) == self.window_size,
            'window_start': self.timestamps[0] if self.timestamps else None,
            'window_end': self.timestamps[-1] if self.timestamps else None
        }
    
    def get_current_ma(self):
        """Get the current moving average without adding new data."""
        if len(self.buffer) == 0:
            return None
        return self.sum / len(self.buffer)
    
    def get_buffer_info(self):
        """Get information about the current buffer state."""
        return {
            'window_size': self.window_size,
            'current_count': len(self.buffer),
            'is_full': len(self.buffer) == self.window_size,
            'current_sum': self.sum,
            'current_ma': self.get_current_ma(),
            'oldest_timestamp': self.timestamps[0] if self.timestamps else None,
            'newest_timestamp': self.timestamps[-1] if self.timestamps else None,
            'price_range': (min(self.buffer), max(self.buffer)) if self.buffer else None
        }
    
    def reset(self):
        """Reset the moving average calculator."""
        self.buffer.clear()
        self.timestamps.clear()
        self.sum = 0.0

# Convenience function for simple usage
def create_streaming_ma(window_size):
    """
    Create a new streaming moving average calculator.
    
    Parameters:
    -----------
    window_size : int
        Number of data points in the moving average window
        
    Returns:
    --------
    StreamingMovingAverage
        New streaming MA calculator instance
    """
    return StreamingMovingAverage(window_size)


# Enhanced Streaming Moving Averages